In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import re

In [35]:
df = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\База Сентябрь 09_25_квартирография.xlsx")

In [34]:
df2 = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\База по годам\2025 новая\02-03.2025_рынок_NEW.xlsx")

In [35]:
df3 = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\База по годам\2025 новая\04.2025_Рынок_легкий.xlsx")

In [36]:
df4 = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\База по годам\2025 новая\05-06.2025_рынок+квартирогр_актуал.xlsx")

In [37]:
df5 = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\База по годам\2025 новая\07-08.2025_рынок.xlsx")

In [38]:
df6 = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\База по годам\2025 новая\Март МО.xlsx")

In [51]:
df7 = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\База по годам\2025 новая\База Январь 2025.xlsx")

In [55]:
df_общий = pd.concat([df, df2, df3, df4, df5, df6, df7], ignore_index=True)

In [36]:
df_общий = df

In [145]:
df_общий = df_общий.drop(columns=['Секция', 'Этаж', 'Номер', 'Месяц'], errors='ignore')

In [37]:
df_общий.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99922 entries, 0 to 99921
Data columns (total 38 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Дата обновления                 99922 non-null  datetime64[ns]
 1   Название проекта                99922 non-null  object        
 2   На англ                         20662 non-null  object        
 3   Промзона                        36001 non-null  object        
 4   Местоположение                  97792 non-null  object        
 5   Метро                           70964 non-null  object        
 6   Расстояние до метро, км         70780 non-null  object        
 7   Время до метро, мин             70215 non-null  object        
 8   Мцк/мцд/бкл                     84435 non-null  object        
 9   Расстояние до мцк/мцд, км       84090 non-null  object        
 10  Время до мцк/мцд, мин           82259 non-null  object        
 11  Бк

In [130]:
# Создадим колонку с месяцем в формате ГГГГ-ММ
df_общий['Месяц'] = df_общий['Дата обновления'].dt.to_period('M')

# Группировка по месяцу и подсчет количества строк
count_by_month = df_общий.groupby('Месяц').size().reset_index(name='Количество строк')

print(count_by_month)

     Месяц  Количество строк
0  2025-01             92689
1  2025-02             91666
2  2025-03            103761
3  2025-04            107415
4  2025-05            109375
5  2025-06            108406
6  2025-07            107811
7  2025-08            106095


In [52]:
df7.columns = df7.columns.str.capitalize().str.strip()

In [12]:
df_общий = df_общий.rename(columns={'Цена лота со ск, руб.': 'Цена со скидкой, руб.'})
df_общий = df_общий.rename(columns={'Цена со ск, руб.': 'Цена со скидкой, руб.'})

### Чистим данные

In [13]:
columns_to_clean = [
    'Расстояние до метро, км',
    'Время до метро, мин',
    'Расстояние до мцк/мцд, км',
    'Время до мцк/мцд, мин',
    'Расстояние до бкл, км',
    'Время до бкл, мин'
]

for col in columns_to_clean:
    df_общий[col] = (
        df_общий[col]
        .astype(str)
        .str.replace(r'[^\d.,]', '', regex=True)
        .str.replace(',', '.', regex=False)
        .replace('', np.nan)
        .astype(float)
    )

In [14]:
mask = (
    (df_общий['Расстояние до метро, км'] > 50) |
    (df_общий['Время до метро, мин'] > 500) |
    (df_общий['Расстояние до мцк/мцд, км'] > 50) |
    (df_общий['Время до мцк/мцд, мин'] > 500) |
    (df_общий['Расстояние до бкл, км'] > 100) |
    (df_общий['Время до бкл, мин'] > 500)
)

df_общий.loc[mask, columns_to_clean] = np.nan

In [54]:
df_общий['Отделка'].unique()

array([nan, 'С отделкой', 'Черновая', 'Без отделки', 'Предчистовая',
       'С отделкой и доп. опциями'], dtype=object)

In [55]:
df_общий['Местоположение'].unique()

array(['МО сред.', 'МО ближ', 'МО дальн.', 'СК/ТТК', 'ТТК/МКАД',
       'за МКАД', 'СК', 'внутри СК', 'ТТК', nan, 'МКАД'], dtype=object)

In [59]:
df_общий['Класс'].unique()

array(['Комфорт', 'Бизнес', 'Премиум', 'Элит', nan, 'Эконом'],
      dtype=object)

In [64]:
df_общий['Статус'].unique()

array(['в продаже', nan, 'продан'], dtype=object)

In [65]:
sorted(x for x in df_общий['Метро'].dropna().unique() if isinstance(x, str))

['Авиамоторная',
 'Автозаводская',
 'Академика янгеля',
 'Академическая',
 'Алексеевская',
 'Алтуфьево',
 'Аминьевская',
 'Аннино',
 'Апрелевка',
 'Арбатская',
 'Аэропорт',
 'Бабушкинская',
 'Багратионовская',
 'Батюнино (2027 г.)',
 'Бауманская',
 'Беговая',
 'Беломорская',
 'Белорусская',
 'Беляево',
 'Бибирево',
 'Библиотека имени ленина',
 'Борисово',
 'Ботанический сад',
 'Братиславская',
 'Бульвар рокоссовского',
 'Бунинская аллея',
 'Бутово',
 'Бутырская',
 'Варшавская',
 'Ватутинки (2030)',
 'Вднх',
 'Верхние лихоборы',
 'Владыкино',
 'Водный стадион',
 'Войковская',
 'Волгоградский проспект',
 'Воробьевы горы',
 'Выставочная',
 'Говорово',
 'Давыдково',
 'Дмитровская',
 'Домодедовская',
 'Дубровка',
 'Зил',
 'Калужская',
 'Кантемировская',
 'Киевская',
 'Китай-город',
 'Кожуховская',
 'Коломенская',
 'Коммунарка',
 'Комсомольская',
 'Коптево',
 'Корниловская',
 'Котельники',
 'Красногвардейская',
 'Краснопресненская',
 'Красные ворота',
 'Кропоткинская',
 'Крылатское',
 'Кузне

In [66]:
df_общий['Тип помещения'].unique()

array(['Квартиры', 'Апартаменты', 'Пентхаус', 'Резиденция', 'Вилла',
       'Ситихаус', 'Таунхаусы', 'Сити-хаус', 'Коттеджи'], dtype=object)

In [67]:
df_общий['Промзона'].unique()

array([nan, 'Свободный земельный участок', 'Нежилое помещение',
       'Свободный участок', 'Промзона', 'Ветхий фонд', 'Нет'],
      dtype=object)

In [68]:
df_общий['Эскроу'].unique()

array(['да', nan, 'нет'], dtype=object)

In [69]:
df_общий['Стадия строительной готовности'].unique()

array([nan, 'благоустройство', 'введен', 'нулевой цикл'], dtype=object)

In [32]:
df_общий['Договор'].unique()

array([nan, 'ДДУ', 'ДКП'], dtype=object)

In [116]:
df_общий[df_общий['Кол-во комнат'] == '5+'][['Название проекта', 'Площадь, кв.м']]

In [145]:
# Условие: строки с "ви а 2 ур 2 ур"
mask = df_общий['Кол-во комнат'] == 'ви а 2 ур 2 ур'

# Заменяем в зависимости от площади
df_общий.loc[mask & (df_общий['Площадь, кв.м'] > 300), 'Кол-во комнат'] = '6e 2 ур'
df_общий.loc[mask & (df_общий['Площадь, кв.м'] <= 300), 'Кол-во комнат'] = '4e 2 ур'

In [33]:
sorted(x for x in df_общий['Кол-во комнат'].dropna().unique() if isinstance(x, str))

['1 2 ур',
 '2 2 ур',
 '2 ур',
 '2е',
 '2е 2 ур',
 '3 ур. 6е',
 '3е',
 '3е 2 ур',
 '4 2 ур',
 '4е',
 '4е 2 ур',
 '5е',
 '5е 2 ур',
 '6е',
 '6е 2 ур',
 '7е',
 '7е 2 ур',
 'mr',
 'Н/Д',
 'пентхаус',
 'студия',
 'таунхаус']

In [76]:
suffixes = [
    '(терр.)', '(терр.+бассейн)', '(терр)', ' терр', ' терр.', ' (антресоль)', ' (патио)', ' city house',
    ' (ситихаус)', '(терр.)', '(терр.)', 'тер.', '(антрессо ь)', 'турраса'
]

taun_variants = ['(таун)', 'таун', 'таунхаус', '(таунхаус)', 'таунхаусы', '(вилла)', 'таенхаес']

level_replacements = {
    'к2L': '2 ур',
    'л': '2 ур',
    'L': '2 ур',
    '2-ур': '2 ур',
    '2 ур.': '2 ур',
    '2 р.': '2 ур',
    '2ур.': '2 ур',
    '2ур': '2 ур',
    'l': '2 ур'
}

def normalize_safe(value):
    """
    Безопасная нормализация поля 'Кол-во комнат':
    - исправляет типовые ошибки в количестве комнат
    - заменяет 'таунхаус', 'пентхаус'
    - не теряет строки, если не удается нормализовать
    """

    if not isinstance(value, str):
        return value
    val = value.strip()
    if not val:
        return value

    # Если это полностью цифры — возвращаем число и прекращаем обработку
    if val.isdigit():
        return int(val)
        
    original = value

    if not isinstance(value, str):
        return value  # уже число или другой тип

    val = value.strip()
    if not val:
        return original  # пустые строки оставляем как есть

    # Спец. случаи
    if val == '6еL (терр.)':
        val = '6еL'

    # Таунхаус
    if any(variant in val for variant in taun_variants):
        return 'таунхаус'

    # Пентхаус
    if 'ph' in val or 'пент.' in val or 'пентхаес' in val or 'пентхаусы' in val:
        return 'пентхаус'

    # Удаляем суффиксы
    for suffix in suffixes:
        if val.endswith(suffix):
            val = val[:-len(suffix)].strip()
            break

    # Заменяем уровни (2 ур и аналоги)
    if 'к2L' in val:
        parts = [p.strip() for p in val.split('к2L') if p.strip()]
        val = ' '.join(parts) + ' 2 ур'

    for variant, replacement in level_replacements.items():
        if variant != 'к2L' and variant in val:
            val = val.replace(variant, replacement)

    # Перенос "2 ур" в конец
    if '2 ур' in val:
        parts = [p.strip() for p in val.split('2 ур') if p.strip()]
        count = val.count('2 ур')
        val = ' '.join(parts) + (' 2 ур' * count if count else '')

    # Общие исправления
    replacements = {
        '4е2 2 ур': '4е 2 ур',
        '2 4e 2 ур': '4e 2 ур',
        '6е (терр.) 2 ур': '6е 2 ур',
        '3к': '3',
        'e': 'е',
        'st2': 'студия',
        '8е3': '8е',
        't': 'е',
        'у': 'е',
        'стедия': 'студия',
        'ер': 'ур',
        '0x2a': 'студия',
        'st': 'студия',
        'sе' : 'студия',
        '0': 'студия',
        'mr': 'многокомн',
        'многокомн.': 'многокомн',
        'таенхаес': 'таунхаус',
        '1студияе 2 ур': 'студия 2 ур',
        'Стедия' : 'студия',
        '32 2 ур' : '3е 2 ур',
        '2к2 2 ур' : '2 2 ур',
        'пентхаес' : 'пентхаус', 
        '3е турраса' : '3е',
        'ви а 2 ур ': '2 ур',
        '  ' : ' ',
        '1-евро' : '1е',
        'евро-2' : '2е',
        'евро-3' : '3е',
        'евро-4' : '4е',
        'евро-5' : '5е',
        'е2' : '2е',
        'е3' : '3е',
        'е4' : '4е',
        'е6' : '6е',
        'е7' : '7е',
        '3 ур. 6е' : '6е 3 ур',
        '3 ур. 9е' : '9е 3 ур' ,
        'две' : '2',
        'одна' : '1',
        'три' : '3',
        'четыре' : '4', 
        's' : 'студия',
        '5+' : 'многокомн',
        'm' : '2'
        
    }

    for old, new in replacements.items():
        val = val.replace(old, new)

    val = val.strip()

    # Попытка привести к числу комнат
    try:
        # если строка состоит только из цифр, вернем число
        if val.isdigit():
            return int(val)
    except:
        pass

    return val if val else original


In [61]:
def remove_parentheses_safe(val):
    if not isinstance(val, str):
        return val
    original = val
    # удаляем содержимое скобок, но сохраняем остальное
    val_clean = re.sub(r'\([^)]*\)', '', val).strip()
    if not val_clean:
        return original  # если пусто, возвращаем исходное
    return val_clean.lower()

df_общий['Кол-во комнат'] = df_общий['Кол-во комнат'].apply(remove_parentheses_safe)

In [77]:
df_общий['Кол-во комнат'] = df_общий['Кол-во комнат'].apply(normalize_safe)
df_общий['Кол-во комнат'] = df_общий['Кол-во комнат'].astype(str).replace('nan', '')

In [70]:
df_общий['Округ'].unique()

array(['Пушкинский', 'Королев', 'Одинцовский', 'Богородский',
       'Красногорский', 'ЮАО', 'ЦАО', 'СВАО', 'САО', 'Химкинский', 'НАО',
       'ЮВАО', 'Ленинский', 'Балашихинский', 'Люберецкий', 'ВАО', 'СЗАО',
       'ЗАО', 'ЮЗАО', 'Подольск', 'ТАО', 'Воскресенск', nan, 'Дубна',
       'ЗелАО', 'Наро-Фоминский', 'Домодедово', 'Дмитровский',
       'Мытищинский', 'Котельники', 'Рузский', 'Реутов', 'Щелковский',
       'Коломна', 'Раменский', 'Истринский', 'Ногинский', 'Волоколамский',
       'Солнечногорский', 'Сергиево-Посадский', 'Клин', 'Луховицкий',
       'Ступино', 'Серпуховский', 'Талдомский', 'Павловский Посад',
       'Лосино-Петровский', 'Долгопрудный', 'Жуковский'], dtype=object)

In [71]:
df_общий['Конструктив'].unique()

array(['Монолит', 'Кирпич', 'Панель', nan, 'Блоки'], dtype=object)

In [72]:
sorted(x for x in df_общий['Район'].dropna().unique() if isinstance(x, str))

['Академический',
 'Алексеевский',
 'Алтуфьевский',
 'Андреевка',
 'Апрелевка',
 'Арбат',
 'Аристово',
 'Аэропорт',
 'Бабушкинский',
 'Балашихинский',
 'Басманный',
 'Беговой',
 'Бескудниковский',
 'Бибирево',
 'Бирюлево Восточное',
 'Бирюлево Западное',
 'Богородское',
 'Булатниковское',
 'Бутырский',
 'Видное',
 'Внуково',
 'Внуковское',
 'Войковский',
 'Волоколамск',
 'Воскресенский',
 'Восточное Дегунино',
 'Восточное Измайлово',
 'Гагаринский',
 'Глухово',
 'Головинский',
 'Гольяново',
 'Горки Ленинские',
 'Горское',
 'Даниловский',
 'Десеновское',
 'Дмитровский',
 'Долгопрудный',
 'Домодедовский',
 'Донской',
 'Дорогомилово',
 'Дубна',
 'Егорово',
 'Жабкино',
 'Жаворонковский',
 'Жаворонковское',
 'Железнодорожный',
 'Жуковский',
 'Замоскворечье',
 'Западное Дегунино',
 'Заречье',
 'Звенигородский',
 'Ивановское',
 'Ивантеевка',
 'Измайлово',
 'Ильинский',
 'Ильинское',
 'Капотня',
 'Климовск',
 'Клин',
 'Клубничное Поле',
 'Клязьма-Старбеево',
 'Кокошкино',
 'Коломна',
 'Коммуна

In [49]:
df_общий["Цена лота, руб."] = pd.to_numeric(
    df_общий["Цена лота, руб."].astype(str).str.replace(r"[^\d,\.]", "", regex=True).str.replace(",", "."),
    errors="coerce"
)
df_общий["Цена со скидкой, руб."] = pd.to_numeric(
    df_общий["Цена со скидкой, руб."].astype(str).str.replace(r"[^\d,\.]", "", regex=True).str.replace(",", ".").str.replace(" ", ""),
    errors="coerce"
)
df_общий["Цена кв.м со ск, руб."] = pd.to_numeric(
    df_общий["Цена кв.м со ск, руб."].astype(str).str.replace(r"[^\d,\.]", "", regex=True).str.replace(",", "."),
    errors="coerce"
)

In [58]:
df_общий["Класс"] = df_общий["Класс"].str.strip().str.capitalize().str.replace('Типовой', 'Стандарт')
df_общий['Статус'] = df_общий['Статус'].str.strip().str.lower().replace('приостановлен', 'продажи приостановлены')
df_общий['Эскроу'] = df_общий['Эскроу'].str.strip().str.lower()
df_общий['Стадия строительной готовности'] = df_общий['Стадия строительной готовности'].str.strip().str.lower()
df_общий['Договор'] = df_общий['Договор'].str.strip().replace('дду', 'ДДУ').replace('бронирование', np.nan).replace('благоустройство', np.nan).replace('в продаже', np.nan)

In [73]:
помещения_dict = {
    'Квартира' : 'Квартиры',
    'Таун' : 'Таунхаусы',
    'Таунхаус' : 'Таунхаусы',
    'Коттедж' : 'Коттеджи'
    
       
}

df_общий['Тип помещения'] = df_общий['Тип помещения'].str.strip().str.capitalize().replace(помещения_dict)

In [40]:
промзона_dict = {
    'Нежилые здания' : 'Нежилое помещение',  
    'Нежилое здание' : 'Нежилое помещение', 
    'Нежилые помещения' : 'Нежилое помещение',
    'Нежилые' : 'Нежилое помещение',
    'Нежлые здания' : 'Нежилое помещение',
    'Жилые здания' : 'Жилое здание',
    'Жилой дом' : 'Жилое здание',
    'Лесопарковая зона' : 'Свободный участок', 
    'Свободная площадка' : 'Свободный участок', 
    'Участок' : 'Свободный участок',
    'Да' : 'Промзона',
    'Промона' : 'Промзона',
    'Свободынй земельный участок' : 'Свободный земельный участок',
    'Ветхое жилье' : 'Ветхий фонд'

    
}

df_общий['Промзона'] = df_общий['Промзона'].str.strip().str.capitalize().replace(промзона_dict)

In [41]:
метро_dict = {
    'Авиамторная' : 'Авиамоторная',
    'Акажемическая' : 'Академическая',
    'Бабшкинская' : 'Бабушкинская',
    'Варшавская (закрыта)' : 'Варшавская',
    'Верхние Лихобры' : 'Верхние Лихоборы',    
    'Днамо' : 'Динамо',  
    'Медветково' : 'Медведково',
    'Мичуринский просрект' : 'Мичуринский проспект',
    'Нахимоовский проспект' : 'Нахимовский проспект',
    'Перовский парк' : 'Петровский парк',
    'Пятницкое шосс' : 'Пятницкое шоссе',
    'Рассказово' : 'Рассказовка',
    'Строгиер' : 'Строгино', 
    'Верхние лихобры' : 'Верхние Лихоборы'
    
}

df_общий['Метро'] = df_общий['Метро'].str.strip().str.capitalize().replace(метро_dict)

In [42]:
заменяемые_значения_опции = [
    'С отделкой, кухней и мебелью', 'С отделкой и мебелью',
    'С отделкой, кухней и техникой', 'С отделкой, мебелью и кухней',
    'С отделкой и кухней', 'C отделкой и мебелью',
    'С отделкой,мебель и техника', 'С отделкой,кухня,мебель',
    'С отделкой и доп.опциями', 'С отделкой, мебелью и техникой', 
    'С отделкой, кухнеи и мебелью', 'С отделкой и  кухней', 
    'С отделкой и кухня', 'С отделкой,кухня', 'С отделкой, сантехникой',
    'С отделкой с кухней и мебелью в ванной', 'С отделкой и кухней и мебелью в ванной',
    'С отделкой и мебелировкой', 'С отделкой,мебель',
    'С отделкой и мебель', 'С отделкой,кухня и техника', 
    'С отделкой, кухней и мбелью', 'С отделкой и кухней,мебель', 
    'С отделкой,кухня,шкафы', 'С мебелью', 'С мебелью и кухней',
    'С отделкой ,мебель', 'С отделкой,кухня и мебель', 'С отделкой мебель',
    'С отделкой,кухня,техника', 'С оделкой, кухней и техникой',
    'С отделкой и кухней без техники', 'С отделкой,кухней и техникой',
    'С отделкой,мебель,техника', 'С отделкой (мебель и техника)', 
    'С отделкой и сантехникой', 'Меблировка', 'Чистовая с мебелью',
    'Modern_style_turnkey_furniture', 'Neoclassical_furniture', 
    'Finishingkitchen', 'Кухня под ключ', 'С отделкой, с меблировкой',
    'С отделкой и допопциями', 'С отделкой, кухней и ванной'
]

заменяемые_значения_отделка = {'С отделкой,машиноместо' : 'С отделкой',
       'Предчистовая,машиноместо' : 'Предчистовая',
        'Без отделки,машиноместо' : 'Без отделки',
       'Предчистовая и м/м' : 'Предчистовая', 
        'С отделкой и м/м' : 'С отделкой',
        'Чистовая' : 'С отделкой',
       'Без отделки, м/м входит' : 'Без отделки',
        'без отделки' : 'Без отделки',
        'с отделкой' : 'С отделкой',
        'Без отделкт' : 'Без отделки',
        'Сотделкой' : 'С отделкой'  ,
        'Предчистова' : 'Предчистовая',
        'Отделка' : 'С отделкой',
        'Нет' : 'Без отделки',
        'Сотделкой' : 'С отделкой',
        'С отделкой и доп опциями' : 'С отделкой и доп. опциями',
        'White box' : 'Предчистовая',
        'Mr base' : 'Предчистовая',
        'Mr ready' : 'С отделкой',
        'White-box' : 'Предчистовая',
        'Предварительная' : 'Предчистовая',
        'Whitebox' : 'Предчистовая',
        'Отделка white box' : 'Предчистовая',
        'Модерн' : 'С отделкой',
        'Классика' : 'С отделкой',
        'Предчистовая отделка' : 'Предчистовая',
        'Дизайнерский ремонт' : 'С отделкой',
        'Да' : 'С отделкой',
        'Чистовая отделка' : 'С отделкой',
        'Modern_style' : 'С отделкой',
        'Дизайнерская отделка' : 'С отделкой',
        'White box max' : 'Предчистовая',
        'Купите сейчас - платите потом' : np.nan,
       'Бонусная программа от гк фск' : np.nan,
        'Живите сейчас - платите потом' : np.nan,
       'Скидка' : np.nan, 'Рассрочка 0%' : np.nan,                           
                                                              
                             }

df_общий['Отделка'] = df_общий['Отделка'].str.strip().str.capitalize()
df_общий['Отделка'] = df_общий['Отделка'].replace(заменяемые_значения_опции, 'С отделкой и доп. опциями')
df_общий["Отделка"] = df_общий["Отделка"].replace(заменяемые_значения_отделка)

In [43]:
монолит = ['Монолит-кирпич', 'Монолит-железобетон', 'Монолит-каркас', 'Монолитно-кирпичный',
           'Монолит-кипич', 'Монолитно-кирпич', 'Монолит кирпич', 'Монолит жб', 
           'Монолит железобетон', 'Монолитный железобетон']
df_общий['Конструктив'] = df_общий['Конструктив'].str.strip().str.capitalize().replace(монолит, 'Монолит')
df_общий['Конструктив'] = df_общий['Конструктив'].replace(r'(?i)^пик[\s\-]?\d*$', 'Пик', regex=True)

In [44]:
округа_dict = {
    'Мытищи': 'Мытищинский',
    'Красногорск': 'Красногорский',
    'Люберцы': 'Люберецкий',
    'Вёшки': 'Вешки',
    'Химки': 'Химкинский',
    'Вешки': 'Мытищинский',
    'Видное' : 'Ленинский'
}

df_общий['Округ'] = df_общий['Округ'].str.strip().str.title().replace(округа_dict)

# список округов, которые нужно сделать заглавными
okruga = ['Нао', 'Тао', 'Юао', 'Ювао', 'Зао', 'Юзао', 'Сзао', 'Свао', 'Вао', 'Сао', 'Цао', 'Зелао']

# приведение к верхнему регистру только для нужных значений
df_общий['Округ'] = df_общий['Округ'].apply(lambda x: x.upper() if x in okruga else x).replace('ЗЕЛАО', 'ЗелАО')

In [45]:
район_dict = {
    'Новомытищинский Пр-Т, 2Б' : 'Мытищинский',
    'Кляьма-Старбеево': 'Клязьма-Старбеево',
    'Ленинский Район': 'Ленинский',
    'Преображенской': 'Преображенское',
    'Хорошёвский': 'Хорошевский',  
    'Некрасовский' : 'Некрасовка'
}

df_общий['Район'] = df_общий['Район'].str.strip().str.title().replace(район_dict)

In [46]:
местоположение_dict = {
    'Садовое кольцо / ТТК' : 'СК/ТТК',
    'Садовое кольцо/ТТК' : 'СК/ТТК',
    'ТТК/СК' : 'СК/ТТК',
    'От ТТК до МКАД' : 'ТТК/МКАД',
    'на МКАД' : 'МКАД',
    'МО ближ' : 'МО ближ.',
    'ТТК / МКАД' : 'ТТК/МКАД',
    'на СК' : 'СК',
    'на ТТК' : 'ТТК',
    'у МКАД' : 'МКАД',
    'От ТТК до МКАД' : 'ТТК/МКАД',
    'МО ближ.' : 'МО ближ',
    'внутри  СК' : 'внутри СК'
}

df_общий['Местоположение'] = df_общий['Местоположение'].str.strip().replace(местоположение_dict)

In [47]:
стадии_dict = {
    'монтаж нижних этажей' : 'монтаж первых этажей',
    'монтаж верхних этажей' : 'монтаж последних этажей',
    'сдан' : 'введен',
     'нулевой цткл' : 'нулевой цикл',
     'монтаж последнихх этажей' : 'монтаж последних этажей',
     'мнтаж средних этажей' : 'монтаж средних этажей',
     'нулевой цил' : 'нулевой цикл',
     'монтаж среднихэтажей' : 'монтаж средних этажей',
      'монтаж послдених этажей' : 'монтаж последних этажей', 
    
}

df_общий['Стадия строительной готовности'] = df_общий['Стадия строительной готовности'].str.strip().replace(стадии_dict).replace('предчистовая', np.nan)



In [50]:
df_общий['Цена кв.м со ск, руб.'] = (df_общий["Цена со скидкой, руб."]/df_общий["Площадь, кв.м"]).round()
df_общий['Цена кв.м, руб.'] = (df_общий["Цена лота, руб."]/df_общий["Площадь, кв.м"]).round()

In [51]:
df_общий['Скидка,%'] = (
    df_общий['Скидка,%']
    .astype(str)                 # в строки
    .str.replace('%', '', regex=False)  # убрать знак процента, если есть
    .str.replace(' ', '', regex=False)  # убрать пробелы
    .str.replace(',', '.', regex=False) # заменить запятую на точку
)

# приводим к float (всё некорректное станет NaN)
df_общий['Скидка,%'] = pd.to_numeric(df_общий['Скидка,%'], errors='coerce')


mask = df_общий['Скидка,%'] > 100

df_общий.loc[mask, 'Скидка,%'] = (
    1 - df_общий.loc[mask, 'Цена со скидкой, руб.'] / df_общий.loc[mask, 'Цена лота, руб.'])

df_общий['Скидка,%'] = df_общий['Скидка,%'].round(3)

In [52]:
df_общий['Дата обновления'] = pd.to_datetime('1899-12-30') + pd.to_timedelta(df_общий['Дата обновления'], unit='D')

TypeError: dtype datetime64[ns] cannot be converted to timedelta64[ns]

In [53]:
df_общий['Старт'] = pd.to_datetime(df_общий['Старт'], errors='coerce')

### Сохраняем в файл после всех манипуляций

In [153]:
df_общий.to_csv(r"C:\Users\m.olshanskiy\Desktop\База Сентябрь 09_25_чистый.xlsx", index=False, encoding='utf-8-sig')

In [75]:
df_общий.to_excel(r"C:\Users\m.olshanskiy\Desktop\База Сентябрь 09_25_чистый.xlsx", index=False)

### Словарь для замены названий

In [151]:
name_dict = {'Сити Микс-Новокосино': 'СитиМикс Новокосино', 'Найс Лофт': 'Найс лофт', 'Cityzen': 'CITYZEN',
             'JOIS': 'Jois',
             'Павелецкая сити': 'Павелецкая Сити',
             'Квартал Серебряный Бор': 'Квартал Серебряный бор', 'Indy Towers ': 'Indy Towers',
             'AFI Park «Воронцовский': 'AFI Park Воронцовский',
             'Квартал «Издание»': 'Квартал Издание',
             'Новая Алексеевская Роща': 'Новая Алексеевская роща', 'Событие-2': 'Событие',
             'Событие-3': 'Событие', 'Событие-4': 'Событие', 'Headliner': 'headliner',
             'ЗИЛАРТ': 'Зиларт', 'Никольский квартал отрада ': 'Никольский квартал',
             'Аист резиденс': 'Аист Резиденс', 'Белая дача парк': 'Белая Дача парк', 'Зелёный парк': 'Зеленый парк',
             'Петровский парк II': 'Петровский парк ',
             'Котельники парк': 'Котельники Парк', 'РУСИЧ Кантемировский': 'Кантемировский',
             'Космический квартал': 'Космический Квартал', 'Цветочные поляны': 'Цветочные Поляны',
             'Nagatino i': 'Nagatino i-Land',
             'Riversky': 'RiverSky', 'Лужники Коллекшн': 'LUZHNIKI', 'Monodom Lake': 'Апарт-комплекс Monodom Lake',
             'Аквилон SIGNAL': 'SIGNAL', 'Аквилон BESIDE': 'BESIDE', 'Аквилон BESIDE 2.0': 'BESIDE 2.0',
             'Аквилон Митино': 'Митино', 'Баланс': 'balance', 'Хайд': 'Hide',
             'Новая Звезда-2': 'Новая Звезда', 'Новая Звезда, II очередь': 'Новая Звезда',
             'Новая Звезда, I очередь': 'Новая Звезда',
             'Мой адрес в Ватутинках': 'В Ватутинках', 'Мой адрес в Зеленограде': 'В Зеленограде',
             'Мой адрес на Амурской': 'На Амурской', 'Мой адрес на Варшавском 139': 'На Варшавском 139',
             'Мой адрес на Дмитровском 89': 'На Дмитровском 89', 'Мой адрес на Клинской': 'На Клинской',
             'Мой адрес на Люблинской 80 к7': 'На Люблинской 80 к7',
             'Мой адрес на Профсоюзной з/у 60/1': 'На Профсоюзной з/у 60/1',
             'Мой адрес на Салтыковской': 'На Салтыковской',
             'Мой адрес на Соколиной Горе': 'На Соколиной Горе', 'Мой адрес на Шелихова 9': 'На Шелихова 9',
             'Мой адрес на Щелковском 90к1': 'На Щелковском 90к1',
             'Мой адрес на Большой Филевской': 'На Большой Филевской',
             'UNO, Соколиная гора': 'UNO. Соколиная гора', 'UNO, Головинские пруды': 'UNO. Головинские пруды',
             'UNO, Горбунова': 'UNO. Горбунова',
             'UNO.Горбунова': 'UNO. Горбунова',
             'Лес': 'Комплекс апартаментов LES', 'Левел Лесной (Микрогород в лесу)': 'Level Лесной',
             'Элевен': 'Дизайнерский дом Eleven', 'Диалог': 'Dialog',
             'Лаврушинский 5': 'Лаврушинский', 'Белый град (Белый мыс)': 'Белый град', 'EVO': 'ЭВО',
             'Сколково Ван (Заречный квартал)': 'Skolkovo one', 'Liberty': 'Либерти', 'Влюберцы апарт-комплекс':
                 'Влюберцы', 'Гостиничный комплекс Аметист': 'Аметист', 'Лесная Отрада 2-я очередь': 'Отрада Лесная',
             'Лесная Отрада 1-я очередь': 'Отрада Лесная', 'Облака 2.0': 'Облака', 'Riga Hills': 'Рига Хиллс',
             'КИТ': 'Кит', 'Золотой, жилой квартал': 'Золотой', 'Саввинская 27 от Level': 'Саввинская 27 ',
             'Саввинская 17 от Level': 'Саввинская 17', 'Клубный дом Full House': 'Full House',
             'Апсайд Тауэрс': 'Upside Towers', 'LUZHNIKI': 'Luzhniki Collection', 'SOLE HILL (Соул хилл)': 'SOLE HILL',
             'На вертолетчиков': 'Мой адрес на Вертолетчиков', 'Комплекс апартаментов «Wellbe': 'Wellbe',
             'Дизайнерский дом Eleven': 'Eleven', "D'ORO MILLE": 'Доро Милле', 'Страна.Озёрная': 'Страна. Озерная',
             'Квартал «Метроном': 'Квартал Метроном', 'Квартал «Издание': 'Квартал Издание',
             'Balance': 'balance', 'Никитский-6': 'Никитский, 6', 'REPUBLIC': 'Republic',
             'Страна.Заречная': 'Страна. Заречная', 'Седьмое небо': '7 небо', 'ЛИТЕЙНЫЙ': 'Литейный',
             'Life-time': 'Life Time',
             'Андреевка Life': 'Андреевка Лайф', 'Береговой-2': 'Береговой 2', 'Весна': 'Весна (Ивантеевка)',
             'ЖК SOLE HILL (ЖК Соул хилл)': 'SOLE HILL', 'Дом-Мегалит в Королёве': 'Дом-Мегалит в Королеве',
             'Домодедово Парк Новый квартал': 'Домодедово парк. Новый квартал',
             'Внуково парк': 'Внуково Парк', 'Героев': 'Квартал Героев', 'МЫС – премиальный загородный проект': 'МЫС',
             'Одинград Квартал Семейный': 'Одинград. Квартал семейный', 'Олива Лесной городок': 'Олива',
             'Олимп 3': 'Олимп-3', 'Подлипки Город': 'Подлипки-Город', 'ПОЛЁТ-Купавна': 'Полет Купавна',
             'Сколково': 'Резиденции Сколково ', 'Пятницкое 58': 'Пятницкое, 58',
             'ул. Советская 18 (Мессиер 18)': 'Советская 18 (Мессиер 18)', 'Солнечный Парк': 'Солнечный Парк 2',
             'Страна.Парковая': 'Страна. Парковая', '31 квартал 3-я очередь': '31 Квартал',
             '31 Квартал 2-я очередь': '31 Квартал', '31 квартал': '31 Квартал', 'MOMENTS': 'Moments',
             'Пречистенка': 'Пречистенка 8', 'Зеленая Вертикаль': 'Зеленая вертикаль',
             'Ривер Парк': 'Riverpark', 'Каштановая роща': 'Каштановая Роща',
             'Новоград монино': 'Новоград Монино', 'АРТ': 'Арт', 'RIVER HOUSE': 'Ривер хаус',
             'Пушкино Град': 'Пушкино град',
             'Видный Берег 2.0': 'Видный берег 2.0', 'ЭЛЬЙОН': 'Эльйон', 'INJOY': 'Injoy',
             'Тессинский, 1': 'Тессинский 1', 'Very на ботанической': 'Very на Ботанической',
             '1-ый Ленинградский': '1-й Ленинградский',
             'Борисоглебское-2': 'Борисоглебское',
             'Квартал Румянцево (Мелисад)': 'Квартал Румянцево',
             'Кронштадский 9': 'Кронштадтский 9',
             'Кронштадсткий 9': 'Кронштадтский 9',
             'Петровский парк 2': 'Петровский парк',
             'Пятницкое, 58 (Перец)': 'Пятницкое, 58',
             'РГ. Октябрьское поле': 'Октябрьское поле',
             'Резиденция Замоскворечье': 'Резиденции Замоскворечье',
                          'Событие-1': 'Событие',
             'Событие-5': 'Событие',
             'Тринити-2': 'Тринити',
             'Тринити-3': 'Тринити',
             'Уно. Головинские пруды': 'UNO. Головинские пруды',
             'Фестиваль парк-2': 'Фестиваль парк',
             '1-ый Химкинский': '1-й Химкинский',
             'Восточный (НДВ)': 'Восточный',
             'Гранд Комфорт-2': 'Гранд Комфорт',
             'Легенда Корнево': 'Легенда Коренево',
             'Одинград Семейный': 'Одинград. Квартал семейный',
             'Одинград. Семейный': 'Одинград. Квартал семейный',
             'Светлый мир. Биополис': 'Светлый мир БиоПолиc',
             'Эко Видное 2.0': 'У реки. Эко Видное 2.0',
             'Форевилль': 'Foreville',
             'Войс Тауэрс': 'Voice Towers',
             'Афи Парк Воронцовский': 'AFI Park Воронцовский',
             'Афи Тауэр': 'Afi Tower',
             'Ривер парк': 'Riverpark',
             'Ривер парк Тауэрс Кутузовский': 'River Park Towers Кутузовский',
             'Виктори Парк Резиденс': 'Victory Park Residences',
             'Сити Микс': 'Citi-Mix',
             'Кэпитал Тауэрс': 'Capital Towers',
             'ОКО': 'МФК ОКО',
             'Скай Хаус': 'Sky House',
             'Ракурс': 'Rakurs',
             'Моментс': 'Moments',
             'Портленд': 'Portland',
             'Соул': 'Soul',
             'Ферст': 'FORST',
             'Культ': 'CULT',
             'Чкалов': 'Дом Chkalov',
             'Левел Академическая': 'Level Академическая',
             'Левел Бауманская': 'Level Бауманская',
             'Левел Звенигородская': 'Level Звенигородская',
             'Левел Мичуринский': 'Level Мичуринский',
             'Левел Нагатинская': 'Level Нагатинская',
             'Левел Нижегородская': 'Level Нижегородская',
             'Левел Причальный': 'Level Причальный',
             'Левел Селигерская': 'Level Селигерская',
             'Левел Стрешнево': 'Level Стрешнево',
             'Левел Южнопортовая': 'Level Южнопортовая',
             'Сити Бэй': 'City Bay',
             'Ситизен': 'CITYZEN',
             'Джойс': 'Jois',
             'Веер': 'VEER',
             'Мира': 'МИRA',
             'МОД': 'Mod',
             'Сет': 'SET',
             'Симфони 34': 'Symphony 34',
             'Слава': 'SLAVA',
             'Форум': 'FORUM',
             'Фэймос': 'Famous',
             'Сторис': 'Stories на Мосфильмовской',
             'Нью Форм  Аминьевское': 'Нью Форм Аминьевское',
             'Логос': 'Logos',
             'Сезар Сити': 'SEZAR CITY',
             'Фулл Хаус': 'Full House',
             'Энитео': 'Eniteo',
             'Твелв': 'Twelve',
             'Эвер': 'Ever',
             'Эра': 'ERA',
             'Шампань': 'Champine',
             'Спрингс': 'Springs',
             'Веспер Кутузовский': 'Vesper Кутузовский',
             'Лаки 1905': 'Lucky',
             'Эхэд': 'AHEAD',
             'Квартал Монс': 'Квартал МОНС',
             'Метроном': 'Квартал Метроном',
             'Аквилон Бисайд': 'BESIDE',
             'Аквилон Бисайд 2.0': 'BESIDE 2.0',
             'Инди Тауэрз': 'Indy Towers',
             'Долгоруковская, 25': 'Долгоруковская 25',
             'Кинетик': 'Kinetik',
             'Академик (Троицк)': 'Академик',
             'Монодом Лэйк': 'Апарт-комплекс Monodom Lake',
             'Билак Коломенское': 'Beluck Коломенское',
             'Май Приорити Басманная': 'MYPRIORITY Басманный',
             'Май Приорити Дубровка': 'MYPRIORITY Дубровка',
             'Май Приорити Мневники': 'MYPRIORITY Мневники',
             'Май Приорити Павелецкая': 'MYPRIORITY Павелецкая',
             'РЕКА': 'Река',
             'Соле Хиллс': 'SOLE HILL',
             'Инджой': 'Injoy',
             'Кутузовград 2': 'Кутузов Град',
             'Риверскай': 'RiverSky',
             'Топхиллс': 'TopHills',
             'Форивер': 'Foriver',
             'Вест Гарден': 'West Garden',
             'Лайф тайм (Онест)': 'Life time',
             'Олл ин Салют': 'Salut',
             'Ай лав - Счастье вместе': 'ilove',
             'Тэйт': 'Tate',
             'Хедлайнер': 'headliner',
             'Кристал': 'CRYSTAL Трилогия',
             'Дом Малевич': 'Дом MALEVICH',
             'Дом МАЛЕВИЧ': 'Дом MALEVICH',
             'Северный порт. Береговые кварталы.': 'Северный порт. Береговые кварталы',
             'Вейв': 'WAVE',
             'Обручева, 30': 'Обручева 30',
             'Кинг энд Санс': 'KING & SONS',
             'Мэйнстрит': 'Mainstreet',
             'Ред7': 'RED7',
             'Уно. Соколиная Гора': 'UNO. Соколиная гора',
             'Вери': 'Very на Ботанической',
             'Эмоушн': 'Emotion',
             'ЭвоПарк Измайлово': 'Evopark Измайлово',
             'ЭвоПарк Сокольники': 'Evopark Сокольники',
             'Наметкин Тауэр': 'Nametkin tower',
             'ФизтехСити': 'Физтехсити',
             'Иртышский квартал (2-й Иртышский)': '2-й Иртышский',
             'Барклая,6': 'Барклая 6',
             'Холланд парк': 'Holland park',
             'Хай Лайф': 'High Life',
             'Лайф Варшавская': 'Life Варшавская',
             'Опус': 'Opus',
             'Прайд': 'Pride',
             'Шифт': 'Shift',
             'Вэлби': 'Wellbe',
             'Дрим Рива': 'Dream Riva',
             'Дрим Тауэрс': 'Dream Towers',
             'Нева Тауэрс': 'NEVA TOWERS',
             'Инсайдер': 'INSIDER',
             'Рашин дизайн дистрикт': 'Russian Design District',
             'Союз': 'СОЮЗ',
             'Нова': 'NOVA',
             'Октябрьская 98 (МиррорЗдание)': 'Октябрьская 98',
             'Стремянный 2 (Амарант)': 'Стремянный 2',
             'Зе Файв': 'Five Towers',
             'Скай Вью': 'SkyView',
             'Проспект 39( Дистрикт 39)': 'Проспект 39',
             'СберСити': 'Сберсити',
             'Восемь кленов': '8 Кленов',
             'ВАУ': 'WOW',
             'Страна Заречная': 'Страна. Заречная',
             'Страна Озерная': 'Страна. Озерная',
             'Экотауэр Крылатское (Крылатская33)': 'Клубный квартал Крылатская 33',
             'Диус': 'DIUS',
             'Деко Резиденс': 'Deco Residence',
             'Вест Тауэр': 'West Tower',
             'Легаси': 'Legacy',
             'Оникс': 'Onyx',
             'Стеллар Сити': 'Stellar City',
             'Амбер Сити': 'Amber City',
             'Роттердам': 'Rotterdam',
             'Сидней Прайм': 'Sydney Prime',
             'Сидней Сити': 'Sydney City',
             'Скай Гарден': 'Sky Garden',
             'Селф': 'Self',
             'Воксхолл': 'Voxhall',
             'Мариинн': 'Mariinn Park',
             'Нагатино айленд': 'Nagatino i-Land',
             'Шагал': 'Shagal',
             'Джи Три Театральный': 'G3 Театральный',
             'Олимп': 'Олимп (Хотьково)',
             'Две столицы': 'Две Столицы',
             'По ул. Макаренко': 'по ул. Макаренко',
             'Белый град': 'Белый Град',
             'Мытищи-Сити': 'Мытищи Сити',
             'Москворецкий': 'Москворецкий (Тучково)',
             'Спутник': 'Sky Sputnik',
             'Мытищи-парк': 'Мытищи Парк',
             'Учебный 9': 'в пер. Учебный',
             'Адрес': 'Adres',
             'Бакеево парк': 'Бакеево Парк',
             'Трилогия CRYSTAL': 'CRYSTAL Трилогия',
             'Дом «А': 'Дом А',
             'tate': 'Tate',
             'в пер. Учебный': 'Солнечный',
             'Резиденция 1864' : '1864 Резиденция',
             'Stone Sokolniki' : 'Сокольники',
             'Жилой квартал "Пушкино Град' : 'Пушкино град',
             'Квартал на воде (Графика на воде)' : 'Квартал на воде',
             'Гармония Парк' : 'Гармония парк'

             }



developer_dict = {'ООО ТК Алладин': 'ТК Алладин',
                  'Pioneer Finance': 'Pioneer',
                  'AFI Development': 'AFI',
                  'ДСК': 'ФСК',
                  'Тройка Рэд': '3-RED',
                  'MR Group': 'MR',
                  'Садовое кольцо ГК' : 'Садовое кольцо ',
                  'Патек Групп': 'ПАТЕК Групп',
                  'AEON': 'Аеон',
                  '494УНР': '494 УНР',
                  'Bestcon': 'Бесткон',
                  'Колди (Coldy)': 'Колди',
                  "Монарх" : 'МонАрх',
                  'Среда ': 'Среда',
                  'SisDevelopment': 'СИСдевелопмент',
                  'STONE': 'Стоун',
                  'Sun Development': 'ГК Sun Development',
                  'Левел': 'Level',
                  'Regions Development': 'Регионы',
                  'ООО «СЗ УНИВЕРСАЛЬ»': 'СЗ Универсаль',
                  'ДСК-1' : 'ФСК',
                  '1-ДСК': 'ФСК',
                  'Крост' : 'КРОСТ',
                  'ГК Березовец': 'Березовец',
                  'Девелопмент 120' : 'Девелопмент 54',
                  'Инвест-Строй' : 'Инвестстрой',
                  'Ругрупп Девелопмент' : 'Альфа Проджект',
                  'BTR Group' : 'БТР Групп',
                  'DOGMA' : 'Догма',
                  'Magnum' : 'Магнум',
                  'RDI' : 'Ар Ди Ай',
                  'Инвестстройкомплекс' : 'Инвестстрой',
                  'Ипотечная компания М-6' : 'Стройтек',
                  'ПроГород' : 'Level',
                  'УК СЕТЬСТРОЙ БАЛАШИХА' : 'Сетьстрой',
                  'Имена' : 'СЗ Элита-Эко',
                  'Град' : 'ГРАД',
                  '3S_Group' : '3S Group',
                  'АФИ' : 'AFI',
                  'Capital Group' : 'Capital group',
                  'Колди' : 'Coldy',
                  'Форма' : 'Forma',
                  'GloraX' : 'Glorax',
                  'Хаттон' : 'Hutton Development',
                  'Хаттон Девелопмент' : 'Hutton Development',
                  'IKON Development' : 'IKON',
                  'Стрелецкая Слобода' : 'INSIGMA',
                  'LEGENDA Intelligent Development' : 'LEGENDA',
                  'Sense Development' : 'Sense',
                  'Текта' : 'Tekta',
                  'Веспер' : 'Vesper',
                  'ВиХолдинг' : 'Ви Холдинг',
                  'ГАЛС' : 'Галс',
                  'Родина Групп' : 'Группа Родина',
                  'ЕВРОФАРМАКОЛ' : 'Еврофармакол',
                  'СЗ Спартак' : 'ПАТЕК Групп',
                  'РГ-ДЕВЕЛОПМЕНТ' : 'РГ',
                  'СТ Тауэрс' : 'Ренессанс',
                  'Холдинг «РСТИ» (Росстройинвест)' : 'РСТИ',
                  'ГК Сумма Элементов' : 'Сумма Элементов',
                  'СЗ ГРАД ПЛАЗА' : 'ГРАД',
                  'Профи Инвест' : 'ПРОФИ-Инвест',
                  'Профи-Инвест' : 'ПРОФИ-Инвест',
                  "Дар" : "DAR",
                  'СЗ Галакс+' : 'Галакс',
                  '494 УНР' : 'УНР 494',
                  'Плюс Девелопмент' : 'Плюс девелопмент',
                  'РГ-ДЕВЕЛОРМЕНТ' : 'РГ',
                  'С3 Ключ' : 'СЗ Ключ',
                  'ЦентрЖилСервис-2010' : 'ЦентрЖилСервис-2012',
                  'ЭКДевелопмент' : 'Э.К. Девелопмент',
                  'Айкон' : 'IKON',
                  'Ферро-Строй' : 'Ферро-строй',
                  'СЗ Полёт' : 'СЗ-Полет',
                  'Ин-Групп (Тула)' : 'Ин-Групп',
                  'ООО «СЗ «ЛИТЕЙНЫЙ.ЛАЙФ»' : 'СЗ ЛИТЕЙНЫЙ.ЛАЙФ',
                  'Квартал Инвестстрой' : 'Квартал-Инвестстрой',
                  'МЕССИЕР-ДЕВЕЛОПМЕНТ' : 'Мессиер-Девелопмент',
                  'Аверус' : 'СЗ АВЕРУС',
                  'Ди Эм Холдинг' : 'ДМ Холдинг',
                  'Риверхаус' : 'СЗ Риверхаус',
                  'СЗ Бест Проект': 'СЗ Бест-Проект',
                  'СЗ Сантерра' : 'Сантерра',
                  'СК Зеленоградский' : 'Сибпромстрой',
                  'СМУ 29' : 'СМУ-29',
                  'Эс Ди Ай' : 'КОМФОРТИНВЕСТ',
                  'ГК Монолит' : 'Монолит',
                  'Сити21' : 'Сити 21 век',
                  'ГК Мега-Мечта' : 'Мега-мечта',
                  'Глоракс' : 'Glorax',
                  'ООО СЗ "ИРИСТОН"' : 'СЗ ИРИСТОН',
                  'Галакс' : 'Галакс+',
                  'Проксима-1' : 'ПРОКСИМА-1',
                  'Спсити' : 'СПСИТИ',
                  'Техноолимпстрой' : 'ТехноСтройОлимп',
                  'Джи Три Илинский' : 'G3 Group',
                  'СЗ ДЖИ ТРИ ИЛЬИНСКИЙ' : 'G3 Group',
                  'BAZA Development' : 'Baza',
                  'Эдельвейс (Москва)' : 'Центрстрой',
                  'СЗ ВЭЛЛБИ' : 'Равновесие Капитал',
                  'Апрелевка С2' : 'СИТ',
                  'СЗ СИЯНИЕ' : 'Строй Сервис',
                  'Мегаполис Груп' : 'Capital group',
                  'Ingrad' : 'Sminex',
                  'Апсайд Тауэрс' : 'СМУ-6',
                  'ANT Development' : 'Ant Development',
                  'АСРесурс' : 'Авиаспецресурс',
                  'ГК Садовое кольцо' : 'Садовое кольцо',
                  'СЗ Третий Рим (Москва)' : 'СЗ Третий Рим',
                  'Инвест Траст' : 'ИнвестТраст',
                  'СК ВЫБОР' : 'Выбор',
                  'ГК Основа' : 'Основа',
                  'Хант-Холдинг' : 'Прайм Лайф',
                  '3SGroup' : '3S Group',
                  'ДСК 1' : 'ФСК',
                  'Gravion' : 'Гравион',
                  'Ikon' : 'IKON',
                  'Level Group' : 'Level',
                  'MAGNUM' : 'Магнум',
                  'MR Group/Левел Групп' : 'Level',
                  'Sezar' : 'Sezar Group',
                  'StMichael' : 'St. Michael',
                  'Tekta Group' : 'Tekta',
                  'Touch' : 'TOUCH',
                  'Unikey' : 'Юникей',
                  'Абсолют Недвижимость' : 'Абсолют',
                  'Апсайд'  : 'Upside Development',
                  'БЭЛ' : 'БЭЛ Девелопмент',
                  'Левел Групп' : 'Level',
                  'Легенда' : 'LEGENDA',
                  'Октобер групп' : 'October Group',
                  'Пионер' : 'Pioneer',
                  'Плюс девелопмент' : 'PLUS Development',
                  'РГ-Девелопмент' : 'РГ',
                  'Родина' : 'Группа Родина',
                  'Самолет Девелопмент' : 'Самолет',
                  'Самолет (МИЦ)' : 'Самолет',
                  'Сезар' : 'Sezar Group',
                  'СЗ 5 Донской' : 'СЗ 5 ДОНСКОЙ',
                  'Смайнекс' : 'Sminex',
                  'Смайнекс (Инград)' : 'Sminex',
                  'СМУ-6 Инвестиции' : 'СМУ-6',
                  'Среда (ПИК)' : 'Среда',
                  'Сумма элементов' : 'Сумма Элементов',
                  'Тач' : 'TOUCH',
                  'Центр Инвест' : 'Центр-Инвест',
                  'Glorax Development' : 'Glorax',
                  'Арс' : 'АРС',
                  'Главстрой Девелопмент' : 'Главстрой',
                  'КОНЦЕПТ ИММО ДЕВЕЛОПМЕНТ' : 'ИММО ДЕВЕЛОПМЕНТ',
                  'Кошелев' : 'Корпорация КОШЕЛЕВ',
                  'Кошелев-Проект' : 'Корпорация КОШЕЛЕВ',
                  'Р Инвест' : 'Р-Инвест',
                  'Садовое Кольцо' : 'Садовое кольцо',
                  'УК Сетьстрой Балашиха' : 'Сетьстрой',
                  'Фонд' : 'Фонд развития территорий',
                  'Тренд-Групп' : 'Тренд групп',
                  'ФСК/Каскад' : 'ФСК',
                  'ПРАЙМ ЛАЙФ' : 'Прайм Лайф',
                  'СЗ БАЛЧУГ ДЕВЕЛОПМЕНТ' : 'СЗ БАЛЧУГ ЭСТЕЙТ',
                  'СЗ ИРИСТОН' : 'Тройка плюс',
                  'Колди' : 'Coldy',
                  '1-Й КРАСНОГВАРДЕЙСКИЙ' : 'Сумма Элементов',
                  'Галакс' : 'Галакс+',
                  'Трастком Девелопмент' : 'Основа',
                  'Plus Development' : 'PLUS Development',
                  'Виртус-Р' : 'Ташир'}

In [142]:
df_общий["Название проекта"] = df_общий["Название проекта"].replace(name_dict)
df_общий["Девелопер"] = df_общий["Девелопер"].replace(developer_dict)